In [13]:
import numpy as np
import pandas as pd


In [14]:
#file1 = open('JazzNetwork.dot', 'r+')
file1 = open('LeagueNetwork.dot', 'r+')
#file1 = open('LesMiserables.dot', 'r+')


def extractDot():
    global nodes
    nodes = []
    #edge values
    global edgv1
    edgv1 = {}
    global header 
    header = ""
    global footer 
    footer = ""
    count = 0
    #Parse line by line, check content, sort accordingly
    for i in file1:
        if len(i) == 1:
            continue
        else:
            i = i.strip()
            if '{' in i:
                header = i
            elif '}' in i:
                footer = i
            #These are edges with weights
            elif '=' in i and '-' in i:
                x = i.split("  ")
                edgv1[x[0]] = x[1].replace(";", "")
            #These are edges without weights
            elif '-' in i and '=' not in i:
                x = i.split(" ")
                u = x[2].split(";")
                if int(x[0]) in edgv1:
                    edgv1[int(x[0])].append(int(u[0]))
                else:
                    edgv1[int(x[0])] = [int(u[0])]
            #These are nodes
            else:
                x = i.split(";")
                nodes.append(int(x[0]))      




In [15]:
def createMatrix():
    global b
    b = (int(max(nodes)) +1)
    #empty adjacency matrix
    global em
    em = np.zeros((b,b))
    #empty edge value matrix
    global ev
    ev = np.zeros((b,b))
    count = 0
    #add axes (nodes vertically & horizontally) to matrix
    for i in range(0, (int(max(nodes)) +1)):
        em[0][count] = i
        em[count][0] = i
        ev[0][count] = i
        ev[count][0] = i
        count += 1



In [16]:
def fillMatrix():
    #print(edgv1)
    tempx = 0
    tempy = 0
    #fill adj matrix & edge value matrix 
    for i in edgv1:
        #print(i)
        #print(type(edgv1[i]))
        if type(edgv1[i]) is str:
            if "-" in str(edgv1[i]):
                l = i.split(" ")
                tempx = int(l[0])
                tempy = int(l[2])
                em[tempx][tempy] = 1
                x = edgv1[i].split('''"''')
                ev[tempx][tempy] = int(x[1])
            elif '[' in str(edgv1[i]):
                #print(edgv1[i])
                l = i.split(" ")
                tempx = int(l[0])
                tempy = int(l[2])
                em[tempx][tempy] = 1
                u = edgv1[i].split("=")
                y = u[1].split("]")
                ev[tempx][tempy] = int(y[0])
        else:
            for j in edgv1[i]:
                em[i][j] = 1




        

In [17]:

def getStats():
        #dict to count no of edges per node
        global edgecount
        edgecount = {}
        #dict to count outgoing weights per node
        global edgeweight
        edgeweight = {}
        #statistics to return
        global stats
        stats = {'aepn':0,'maxepn':0,'minepn':0,'aew':0,'maxew':0,'minew':0}
        for i in edgv1.keys():
                if type(i) == str:                
                        o = i.split(" ")
                        #print(o)
                        try:
                                edgecount[int(o[0])] += 1
                                #print(int(o[0]))
                                #print(edgv1[i])
                                if "-" not in edgv1[i]:
                                        g = edgv1[i].split("=")
                                        #print(g)
                                        j = g[1].split("]")
                                        #print(j)
                                        edgeweight[int(o[0])] += int(j[0])
                                else:
                                        g = edgv1[i].split("=")
                                        #print(g)
                                        x = g[1].split('''"''')
                                        #print("X",x)
                                        #print(x)
                                        edgeweight[int(o[0])] += int(x[1])
                        except KeyError:
                                edgecount[int(o[0])] = 1 
                                edgeweight[int(o[0])] = 0
                                
                else:
                        try:
                                edgecount[i] = len(edgv1[i])

                        except KeyError:
                                edgecount[i] = 1 

        stats['aepn'] = round(sum(edgecount.values()) / len(edgecount))
        stats['maxepn'] = max(edgecount.values())
        stats['minepn'] = min(edgecount.values())
        

        if stats['aew'] != 0:
                stats['aew'] = round(sum(edgeweight.values()) / len(edgeweight))
                stats['maxew'] = max(edgeweight.values())
                stats['minew'] = min(edgeweight.values())   
        else:
                print('No edge weights.')

        
x = extractDot()
p = createMatrix()
o = fillMatrix()
z = getStats()




No edge weights.


In [18]:
from collections import defaultdict

class CycleChecker():
    #initialize class variables
    def __init__(self, nodes):
        self.graph = defaultdict(list)
        self.V = nodes
    
    #add edges to graph
    def addEdge(self, b, a):
        self.graph[b].append(a)
 
    #tracking module
    def isCyclicUtil(self, a, checked_nodes, tracker):
 
        #indicate node checked
        checked_nodes[a] = True
        tracker[a] = True
 
        #iterate over all neighbours & neighbours neighbours etc.
        for neighbour in self.graph[a]:
            if checked_nodes[neighbour] == False:
                if self.isCyclicUtil(neighbour, checked_nodes, tracker) == True:
                    return True
            elif tracker[neighbour] == True:
                return True
 
        tracker[a] = False
        return False
 
    #performs Cycle check
    def isCyclic(self):
        checked_nodes = [False] * (self.V + 1)
        tracker = [False] * (self.V + 1)
        for node in range(self.V):
            if checked_nodes[node] == False:
                if self.isCyclicUtil(node, checked_nodes, tracker) == True:
                    return True
        return False
 
 
def checkCycle():
    g = CycleChecker(len(em))
    for i in range(1, len(em)):
        for j in range(1, len(em[i])):
            if em[i][j] > 0:
                g.addEdge(i, j)
            else:
                continue


    if g.isCyclic() > 0:
        print("Graph contains 1 or more cycles")
    else:
        print("Graph doesn't contain cycles")

x = checkCycle()


Graph contains 1 or more cycles
